In [1]:
import scipy.io
import numpy as np
from scipy.optimize import linprog

mat = scipy.io.loadmat('messageFromAlice.mat')

In [3]:
import numpy as np
from scipy.optimize import linprog

# Пример данных (замените на реальные данные задачи)
p = 856  # Размерность исходного сообщения
m = 4 * p  # Размерность зашумленного сообщения

# Случайная матрица кодирования (замените на реальную матрицу A)
# A = np.random.randn(m, p)
A = mat['A']

# Случайное зашумленное сообщение (замените на реальное y')
# y_prime = np.random.randn(m)
y_prime = mat['yprime']
# Размерность переменных: p для x' и m для u
c = np.hstack([np.zeros(p), np.ones(m)])  # Целевая функция

# Ограничения: Ax' - u <= y' и -Ax' - u <= -y'
A_ub = np.vstack([
    np.hstack([A, -np.eye(m)]),  # Ax' - u <= y'
    np.hstack([-A, -np.eye(m)])  # -Ax' - u <= -y'
])
b_ub = np.hstack([y_prime, -y_prime])

# Ограничения на переменные: 0 <= x' <= 1, u >= 0
bounds = [(0, 1)] * p + [(0, None)] * m

# Решение задачи
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Полученные значения переменных x' и u
x_prime = result.x[:p]
u = result.x[p:]

# Вывод результатов
print("Восстановленное сообщение x':", x_prime)
print("Значения переменных u:", u)
print("Успешность решения:", result.success)
print("Целевая функция:", result.fun)


Восстановленное сообщение x': [ 1.04195914e-13  1.00000000e+00 -1.02193785e-13  1.00000000e+00
  1.00000000e+00  1.98825240e-13 -6.33116081e-14  1.00000000e+00
  0.00000000e+00  1.00000000e+00  1.00000000e+00 -0.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
 -3.13147983e-13  1.00000000e+00  1.00000000e+00  1.00000000e+00
  0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00
 -9.66447162e-14 -9.20818169e-14  1.00000000e+00 -2.01726315e-13
  9.09694310e-14  0.00000000e+00  0.00000000e+00  1.54122808e-13
  0.00000000e+00  1.00000000e+00  1.00000000e+00  3.98794242e-14
  6.53780414e-14  0.00000000e+00  1.00000000e+00  1.00000000e+00
  2.18748609e-14  1.00000000e+00  1.00000000e+00  0.00000000e+00
  4.97804753e-14  0.00000000e+00 -2.76194071e-14  1.00000000e+00
  0.00000000e+00  1.00000000e+00  1.00000000e+00  2.60706561e-13
  1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00
  1.66162752e-14  2.63617913e-13  1.00000000e+00  0.00000000

In [13]:
bits = np.where(x_prime < 0.5, 0, 1)
print(bits)

# Конвертация битов в байты (группировка по 8 битов)
byte_array = np.packbits(bits)

# Декодирование байтов в строку
decoded_message = byte_array.tobytes().decode('utf-8', errors='ignore')

print("Декодированное сообщение:", decoded_message)

Декодированное сообщение: You can claim your personal reward by going to Student affairs, giving you code=1120 and ask for you reward


In [2]:
x_true = b'Mr. Leplat, please give us A grade for this project!'

# Преобразование сообщения в байты, затем в биты
x_bytes = np.frombuffer(x_true, dtype=np.uint8)
x_bits = np.unpackbits(x_bytes)

print("Биты сообщения:", x_bits)
x_bits = x_bits[:,np.newaxis]

Биты сообщения: [0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1
 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0
 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 0 0 1 0
 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 0 1 0 1 0
 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1
 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1
 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1
 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1
 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 0
 0 0 0 1 0 0 0 0 1]


In [3]:
A_new = mat['A'][:x_bits.shape[0]*4, :x_bits.shape[0]]
A_new.shape

(1664, 416)

In [4]:
y = A_new @ x_bits
y.shape

(1664, 1)

In [5]:
# Функция для добавления шума
def add_noise(bits, noise_level):
    noisy_bits = bits.copy()
    num_noisy_bits = int(noise_level * len(bits))
    noise_indices = np.random.choice(len(bits), num_noisy_bits, replace=False)
    noisy_bits[noise_indices] = 1 - noisy_bits[noise_indices]  # Инвертируем биты
    return noisy_bits


In [6]:
# Случайная матрица кодирования (замените на реальную матрицу A)
# A = np.random.randn(m, p)
A = A_new.copy()

# Пример данных (замените на реальные данные задачи)
p = x_bits.shape[0]  # Размерность исходного сообщения
m = 4 * p  # Размерность зашумленного сообщения

In [10]:
for noise_level in np.arange(0.35, 0.45, 0.01):
    print("noise: ", noise_level)
    y_noisy_bits = add_noise(y, noise_level).T

    print("Сообщение с шумом:", y_noisy_bits)

    # Случайное зашумленное сообщение (замените на реальное y')
    # y_prime = np.random.randn(m)
    y_prime = y_noisy_bits.copy()
    # Размерность переменных: p для x' и m для u
    c = np.hstack([np.zeros(p), np.ones(m)])  # Целевая функция

    # Ограничения: Ax' - u <= y' и -Ax' - u <= -y'
    A_ub = np.vstack([
        np.hstack([A, -np.eye(m)]),  # Ax' - u <= y'
        np.hstack([-A, -np.eye(m)])  # -Ax' - u <= -y'
    ])
    b_ub = np.hstack([y_prime, -y_prime])

    # Ограничения на переменные: 0 <= x' <= 1, u >= 0
    bounds = [(0, 1)] * p + [(0, None)] * m

    # Решение задачи
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

    # Полученные значения переменных x' и u
    x_prime = result.x[:p]
    u = result.x[p:]

    # Вывод результатов
    print("Восстановленное сообщение x':", x_prime)
    print("Значения переменных u:", u)
    print("Успешность решения:", result.success)
    print("Целевая функция:", result.fun)


    bits = np.where(x_prime < 0.5, 0, 1)
    print(bits)
    # print("Процент несовпадения побитово: ", (bits != x_bits).sum()/x_bits.shape[0])
    # Конвертация битов в байты (группировка по 8 битов)
    byte_array = np.packbits(bits)

    # Декодирование байтов в строку
    decoded_message = byte_array.tobytes().decode('utf-8', errors='ignore')

    print("Декодированное сообщение:", decoded_message)
    print("Процент несовпадения посимвольно: ", (np.frombuffer(x_true, dtype=np.int8) != byte_array).sum() / len(x_true))


Сообщение с шумом: [[ 3.34461373 12.28486606 10.97469087 ... -0.51164786 -4.87877752
   5.80322681]]
Восстановленное сообщение x': [ 0.00000000e+00  9.99999999e-01  0.00000000e+00  2.83884118e-09
  9.99999999e-01  9.99999999e-01 -1.54391821e-09  1.00000000e+00
  0.00000000e+00  1.00000000e+00  1.00000000e+00  9.99999999e-01
  4.08140477e-10 -1.57046638e-09  1.00000000e+00 -1.69260941e-09
  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00
  6.90851887e-10  3.85616471e-10  1.00000000e+00  0.00000000e+00
 -1.21765268e-09  1.50312353e-09  1.13040164e-09  0.00000000e+00
  5.73784303e-10  1.00000000e+00 -7.79477316e-11 -6.67703599e-10
  1.00000000e+00  1.00000000e+00  1.03373573e-09  0.00000000e+00
  0.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00
  0.00000000e+00  1.00000000e+00  7.83511781e-10  1.00000000e+00
  0.00000000e+00  1.00000000e+00  9.99999997e-01  1.00000000e+00
  0.00000000e+00 -1.2636

KeyboardInterrupt: 